In [1]:
%matplotlib qt5

In [2]:
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass
import pyaccel.optics as pyop

In [21]:
# this function redifines the vchamber height 
# after calling this func. the hmax and hmin will be symetric

# essa função bem que podia passar um parametro que faz com que a altura da camara de vacuo nesses indices especificos voltem ao que eram antes 

def set_hchmb(acc, strin, value):
    indices = pyaccel.lattice.find_indices(acc, 'fam_name', strin)
    for iten in indices:
        acc[iten].hmax = value
        acc[iten].hmin = -value
    
    return None

# this function returns to nominal value for vacum chamber height


def nom_hchmb(acc, strin):
    model = pymodels.si.create_accelerator()
    hmax = pyaccel.lattice.get_attribute(model, 'hmax', indices='open')
    hmin = pyaccel.lattice.get_attribute(model, 'hmin', indices='open')
    indices = pyaccel.lattice.find_indices(model, 'fam_name', strin)
    for iten in indices:
        acc[iten].hmax = hmax[iten]
        acc[iten].hmin = hmin[iten]
    
    return None

def track_mchn_stdy(acc, bunch, param, increment, index, nturn):
    # setting param to be the index of the array [x,x',y,y',delta,z]

    bunch[param] += increment
    tracked = pyaccel.tracking.ring_pass(acc, particles=bunch,
                                         nr_turns=nturn, turn_by_turn= True,
                                         element_offset= index, parallel=True)
    
    part_out, lost_flag, turn_lost, index_lost, plane_lost = tracked

    turnl_element = []

    for i,iten in enumerate(turn_lost):
        if iten == nturn and index_lost[i] == index: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((iten, index_lost[i]))
    
    return turnl_element

def varying_incmnts(acc,bunch,par,rng, index, nturn, val, strin, value):

    ''' 
        acc: accelerator 
        bunch: particle initial conditions
        par: index of [x,x',y,y',delta,z] -> [0,1,2,3,4,5]
        rng: range of the increment
        nturn: number of turns
        val: variable that goes in numpy linspace, val is the value till the variation must be performed
        strin: is a string, this string represents the fam_name of the scraper 'SHVC' or 'SVVC'
        value: is the value that the user desires for setting the height of vchamber
    '''

    trackf_lostpos_nom = []
    trackf_lostpos_mod = []
    inc = np.linspace(0, val, rng)
    incdif = np.diff(inc)

    # primeiramente a simulação vai ser realizada com a altura da camara de vacuo com o modelo nominal
    for iten in incdif:
        trackf_lostpos_nom.append(track_mchn_stdy(acc, bunch, par, increment=iten, index=index, nturn=nturn))

    set_hchmb(acc, strin, value) # after calling this function, vchamber's height will be changed
    for iten in incdif:
        trackf_lostpos_mod.append(track_mchn_stdy(acc, bunch, par, increment=iten, index=index, nturn=nturn))

    return trackf_lostpos_nom, trackf_lostpos_mod



def s_data_treat(l_of_tples, idx, val):
    
    size = len(l_of_tples)
    inc = np.linspace(0,val, size)
    for c, iten in enumerate(l_of_tples):
        if len(iten) == 0:
            pass
        else:
            j = c
            break

    if len(l_of_tples[idx]) == 0:
        print("all the electrons survived")
        return

    else:
        nt_lost, index = np.zeros(len(l_of_tples[idx])), np.zeros(len(l_of_tples[idx]))

        for j,iten in enumerate(l_of_tples[idx]):
            nt_lostj, idx_lost = iten
            nt_lost[j] = nt_lostj
            index[j] = idx_lost
        index = np.intp(index)

    ind = np.arange(index.size)


    return c, inc[c], ind, nt_lost

def c_data_treat(tup, idx, val):

    all_data = []
    for iten in tup:
        resul_tup = s_data_treat(iten, idx, val)

        c, inc_c, ind, nt_lost = resul_tup
        all_data.append(ind)
        print(c, inc_c)

    return all_data

# para montar a função que vai realizar o plot dos gráficos podemos simplesmente passar a função acima em um for

def plot_tracked(acc, spos, l_all_data): # é bom que essa função receba os valores das lista de indices onde as paticulas foram perdidas

    nom_track = l_all_data[0] # keep in mind that nominal_tracking is the list of indices that the electrons were lost in the nominal_model with vchamber's height defined by default
    mod_track = l_all_data[1]

    fig, a1, a2 = plt.subplots((1,2), figsize=(10,5), sharey=True, gridspec_kw={'width_ratios':[1,3]})

    a1.set_title(r'título_primário', fontsize=16)
    a2.set_title(r'título_primário', fontsize=16)

    a1.set_xlabel(r'x1', fontsize=14)
    a1.set_ylabel(r'y1', fontsize=14)
    
    a2.set_xlabel(r'x2', fontsize=14)
    a2.set_ylabel(r'y2', fontsize=14)

    a1.scatter(spos[nom_track],nom_track, s=4)#, label='perde a partir de {}'.format(inc[c]))
    a2.scatter(spos[mod_track],mod_track, s=4)#, label='perde a partir de {}'.format(inc[c]))

    






# plt.figure()
# plt.title('electron losses by x parameter variation', fontsize=16) # . incremento {:.3f}'.format(inc[idx])
# plt.scatter()
# pyaccel.graphics.draw_lattice(acc, offset=0, height= 1, gca=True)
# plt.legend()

# plt.xlabel(r's position [m]', fontsize=14)
# plt.ylabel(r'# lost electron', fontsize=14)


# # plt.xlim(0, spos[index][-1])
# # plt.plot(spos[np.intp(index)], nt_lost)
# plt.show()




# the objective is make a comparison between tracking simulation with and without varying the vaccum chamber height

# agora falta criar a (ou as) função que vai realizar essa comparação

In [17]:
fitm = pymodels.si.create_accelerator()
fitm = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fitm)
eqparams = pyaccel.optics.beam_envelope.EqParamsFromBeamEnvelope(fitm)

# parameters to generate the beam
h_emitt = eqparams.emit1
v_emitt = h_emitt/100
sigmae = eqparams.espread0
bun_len = eqparams.bunlen

# defining optcis
twiss,*_ = pyop.calc_twiss(fitm, indices='closed')

# defining the index 
dicty = pyaccel.lattice.find_dict(fitm, 'fam_name')
nlk_index = dicty['InjNLKckr'][0] + 1
spos = pyaccel.lattice.find_spos(fitm, indices='open')

# generating gaussian beam.
bunch = pyaccel.tracking.generate_bunch(50, envelope=None, emit1=h_emitt,
                                emit2=v_emitt, sigmae= sigmae, sigmas=bun_len, optics=twiss[nlk_index])


In [5]:
variable = varying_incmnts(fitm, bunch[:,0:20], 0, 50, nlk_index, 7, 0.01)

In [11]:
spos = pyaccel.lattice.find_spos(fitm, indices='closed')

cont, lins, index, ntlost = plot_tracked(fitm, variable, 48, spos, 0.01)


In [12]:
# selecionando os indices que correspondem aos scrapers
pyaccel.lattice.find_indices(fitm, 'fam_name', 'SHVC')

[6532, 6536]

In [ ]:
set_hchmb(fitm, 'SHVC', 0.09)



In [ ]:
fitm[6532].hmin

In [ ]:
# # anotações que eu pensei que pudessem ser importantes

# # definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe
# # eu não sei exatamente se essa celula no meu código é util, porque em um primeiro momento vamos variar apenas
# # as posições x, y, x' e y'

# fit_acc = pymodels.si.create_accelerator()
# fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
# tousan = tousclass.Tous_analysis(fit_acc)
# deltas = tousan.deltas
# n_turns = tousan.nturns

# orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
# orb = orb[:, 1]
# np.zeros((6, deltas.size)), orb[:, None]

# rin = np.zeros((6, deltas.size))
# rin += orb[:,None]

# mean = np.zeros(6)
# env = pyop.beam_envelope.calc_beamenvelope(fit_acc)
# cov = env[nlk_index]
# particles = np.random.multivariate_normal(mean, cov, size=1).T

# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()

# # esse procedimento é feito para ir variando os parâmetros de posição 
# vecy = np.linspace(4.8332669321064365e-09, 4.8332669321064365e-02, 1000)
# vecx = np.linspace(2.25702434e-08,2.25702434e-02, 1000)
# # com a função vstack do numpy podemos juntar dois arrays e formar uma matriz
# np.vstack((vecx, vecy))